# Data Engineering Capstone Project

## Enviroment setup

In [1]:
# Import necessary libraries
import pandas as pd
import uuid
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import monotonically_increasing_id as mono_id
import configparser

In [2]:
# Read config file
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

INPUT_DATA = config['LOCAL']['INPUT_DATA']
INPUT_DATA_VACCINES = config['LOCAL']['INPUT_DATA_VACCINES']
OUTPUT_DATA = config['LOCAL']['OUTPUT_DATA']
DATA_COLUMNS = config['COMMON']['DATA_COLUMNS']

In [3]:
@udf(StringType())
def null_id_to_uuid (id):
    if id == None or id == "null":
        return str(uuid.uuid4().hex)
    return id

In [4]:
def write_parquet(df, parquet_name):
    parquet_path = OUTPUT_DATA + f'{parquet_name}.parquet'
    df.write.mode("overwrite").parquet(parquet_path)
    print(f'Writing {parquet_name} Table DONE.')


In [5]:
def read_parquet(parquet_name):
    parquet_path = OUTPUT_DATA + f'{parquet_name}.parquet'
    return spark.read.parquet(parquet_path)

In [6]:
def check_nulls(df, columns_list, expected_value):
    df.createOrReplaceTempView("viewcheck")
    sql_check = f"SELECT COUNT(*) FROM viewcheck WHERE 1 <> 1 {''.join([' OR ' + c + ' IS NULL ' for c in columns_list])}"
    
    dfcheck = spark.sql(sql_check)
    
    value_check = dfcheck.collect()[0][0]
    
    return value_check == expected_value

In [7]:
def check_has_content(df):
    return df.count() > 0

In [8]:
# Spark session
spark = SparkSession \
        .builder\
        .appName("Brazilian COVID-19 Immunization")\
        .getOrCreate()

## Step 1: Scope the Project and Gather Data
In this step, we’ll:

* Identify and gather the data we'll be using for our project (at least two sources and more than 1 million rows).
* Explain what end use cases we'd like to prepare the data for (e.g., analytics table, app back-end, source-of-truth database, etc.)

We choose the following datasets:
* Brazilian Government' dataset [COVID-19 population immunization program](https://dados.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8?inner_span=True)
    * Complete data: "Dados completos"
    * Data per Brazilian States: "Dados XX", where XX is the State code

For development we include just a subset of data in /data/vaccines.csv.

When executing the pipeline on production, we should download the complete dataset, replacing the file vaccines.csv.

**The dataset will be prepared for a data warehouse analysis table, available for public use (citizens, press, health institutions), for monitoring population immunization rates.**.

In [9]:
vaccines_df = spark.read.csv(INPUT_DATA_VACCINES, sep=';', header=True)

vaccines_df.printSchema()

root
 |-- data_importacao_rnds: string (nullable = true)
 |-- document_id: string (nullable = true)
 |-- estabelecimento_municipio_codigo: string (nullable = true)
 |-- estabelecimento_municipio_nome: string (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estabelecimento_uf: string (nullable = true)
 |-- estabelecimento_valor: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = true)
 |-- id_sistema_origem: string (nullable = true)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_endereco_cep: string (nullable = true)
 |-- paciente_endereco_coibgemunicipio: string (nullable = true)
 |-- paciente_endereco_copais: string (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = true)
 |-- paciente_endereco_nmpais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- paciente_enumsexobiologico: string (nullable = true)
 |-- paciente_id: string (nullable = true)
 |-- pac

## Step 2: Explore and Assess the Data
In this step we need:
* Explore the data to identify data quality issues, like missing values, duplicate data, etc.
* Document steps necessary to clean the data

In [10]:
# Read the data dictionary from JSON and extract the valid columns
col_names = pd.read_json(DATA_COLUMNS, typ='series')
valid_columns = col_names.index
valid_columns

Index(['paciente_id', 'paciente_idade', 'paciente_datanascimento',
       'paciente_enumsexobiologico', 'paciente_endereco_nmpais',
       'paciente_endereco_uf', 'paciente_endereco_nmmunicipio',
       'estabelecimento_razaosocial', 'estalecimento_nofantasia',
       'estabelecimento_uf', 'estabelecimento_municipio_nome',
       'vacina_categoria_codigo', 'vacina_categoria_nome',
       'vacina_grupoatendimento_codigo', 'vacina_grupoatendimento_nome',
       'vacina_fabricante_nome', 'vacina_codigo', 'vacina_nome',
       'vacina_dataaplicacao', 'vacina_descricao_dose'],
      dtype='object')

In [11]:
# Get the difference between the dataframe colums and the valid columns
columns_todrop = list(set(vaccines_df.columns) - set(valid_columns))

columns_todrop

['sistema_origem',
 'data_importacao_rnds',
 'estabelecimento_valor',
 'paciente_endereco_cep',
 'paciente_nacionalidade_enumnacionalidade',
 'estabelecimento_municipio_codigo',
 'vacina_fabricante_referencia',
 'vacina_lote',
 'paciente_endereco_coibgemunicipio',
 'paciente_racacor_valor',
 'document_id',
 'paciente_racacor_codigo',
 'paciente_endereco_copais',
 'id_sistema_origem']

In [12]:
# Remove unused columns from dataframe
vaccines_df = vaccines_df.drop(*columns_todrop)
vaccines_df.printSchema()

root
 |-- estabelecimento_municipio_nome: string (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estabelecimento_uf: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = true)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = true)
 |-- paciente_endereco_nmpais: string (nullable = true)
 |-- paciente_endereco_uf: string (nullable = true)
 |-- paciente_enumsexobiologico: string (nullable = true)
 |-- paciente_id: string (nullable = true)
 |-- paciente_idade: string (nullable = true)
 |-- vacina_categoria_codigo: string (nullable = true)
 |-- vacina_categoria_nome: string (nullable = true)
 |-- vacina_codigo: string (nullable = true)
 |-- vacina_dataaplicacao: string (nullable = true)
 |-- vacina_descricao_dose: string (nullable = true)
 |-- vacina_fabricante_nome: string (nullable = true)
 |-- vacina_grupoatendimento_codigo: string (nullable = true)
 |-- vacina_grupoatendi

In [13]:
# Replace the null values
vaccines_df = vaccines_df\
    .withColumn('paciente_id_null', null_id_to_uuid(vaccines_df.paciente_id))\
    .drop('paciente_id')\
    .withColumnRenamed('paciente_id_null', 'paciente_id')
    
vaccines_df = vaccines_df.fillna(\
    {\
        'vacina_categoria_codigo': 0, \
        'vacina_categoria_nome': 'N/A', \
        'vacina_grupoatendimento_nome': 'N/A', \
        'paciente_enumsexobiologico': 'N/A',\
        'paciente_endereco_nmmunicipio': 'N/A', \
        'paciente_endereco_nmpais': 'N/A', \
        'paciente_endereco_uf': 'N/A', \
        'estalecimento_nofantasia': 'N/A'
    })

In [14]:
vaccines_df.printSchema()

root
 |-- estabelecimento_municipio_nome: string (nullable = true)
 |-- estabelecimento_razaosocial: string (nullable = true)
 |-- estabelecimento_uf: string (nullable = true)
 |-- estalecimento_nofantasia: string (nullable = false)
 |-- paciente_datanascimento: string (nullable = true)
 |-- paciente_endereco_nmmunicipio: string (nullable = false)
 |-- paciente_endereco_nmpais: string (nullable = false)
 |-- paciente_endereco_uf: string (nullable = false)
 |-- paciente_enumsexobiologico: string (nullable = false)
 |-- paciente_idade: string (nullable = true)
 |-- vacina_categoria_codigo: string (nullable = false)
 |-- vacina_categoria_nome: string (nullable = false)
 |-- vacina_codigo: string (nullable = true)
 |-- vacina_dataaplicacao: string (nullable = true)
 |-- vacina_descricao_dose: string (nullable = true)
 |-- vacina_fabricante_nome: string (nullable = true)
 |-- vacina_grupoatendimento_codigo: string (nullable = true)
 |-- vacina_grupoatendimento_nome: string (nullable = false

In [15]:
# Write staging table and read back
write_parquet(vaccines_df, 'staging_immunization')

vaccines_df = read_parquet('staging_immunization')

Writing staging_immunization Table DONE.


## Step 3: Define the Data Model
_Map out the conceptual data model and explain why you chose that model_

The data model is a star schema consisting of 5 Dimensions table and 1 Fact table:
  * Dimensions tables:
      * vaccines table: Vaccines and suppliers
      * health_institution table: Hospitals, Nursing home, Clinics 
      * category table: Priority groups
      * population_group table: Demograph group (professions, age group, ethnicity)
      * patient table: Demograph data (age, city, gender)
  * Fact table:
      * immunization table: Dimensions, First | second dose, date

![ER Data Model - Star Scheme](./docs/er-model-star.jpg)

_List the steps necessary to pipeline the data into the chosen data model_
* ETL starts the enviroment setup: imports, read config file, def functions and create Spark Session
* ETL script takes source data (Brazilian Government' dataset COVID-19 population immunization program)
* Raw data is read into dataframe and cleaned (remove unused columns, fill nulls) 
* For each dimension and fact table 
	* Create a temporary view table
	* Read data to new dataframe
    * Check data quality: key columns don't have nulls, each table has content
	* Create id/indexes (if necessary)
	* Write parquet files

## Step 4: Run ETL to Model the Data
* Create the data pipelines and the data model
* Include a data dictionary
* Run data quality checks to ensure the pipeline ran as expected
	* Integrity constraints on the relational database (e.g., unique key, data type, etc.)
	* Unit tests for the scripts to ensure they are doing the right thing
	* Source/count checks to ensure completeness

In [16]:
# Create vaccines table
vaccines_df.createOrReplaceTempView("vaccines_table_DF")
vaccines_table_DF = spark.sql("""
    SELECT  DISTINCT vacina_codigo AS id, 
                     vacina_nome AS name, 
                     vacina_fabricante_nome AS supplier
    FROM vaccines_table_DF
    ORDER BY supplier
""")

vaccines_table_DF.printSchema()
vaccines_table_DF.show()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- supplier: string (nullable = true)

+---+--------------------+--------------------+
| id|                name|            supplier|
+---+--------------------+--------------------+
| 86|Covid-19-Coronava...|   FUNDACAO BUTANTAN|
| 85|Vacina Covid-19 -...|FUNDACAO OSWALDO ...|
+---+--------------------+--------------------+



In [17]:
# Check data quality
if not check_nulls(vaccines_table_DF, ['id', 'name', 'supplier'], 0): raise Exception('Null: Vaccines tables')
if not check_has_content(vaccines_table_DF): raise Exception('No content: Vaccines table')

In [18]:
# Write parquet file
write_parquet(vaccines_table_DF, 'vaccines')

Writing vaccines Table DONE.


In [19]:
# Create Health Institution table
vaccines_df.createOrReplaceTempView("health_institution_table_DF")
health_institution_table_DF = spark.sql("""
    SELECT DISTINCT estalecimento_nofantasia AS name,
                    estabelecimento_razaosocial AS organization,
                    estabelecimento_uf AS state,
                    estabelecimento_municipio_nome AS city
    FROM health_institution_table_DF
    ORDER BY name
""")

health_institution_table_DF = health_institution_table_DF.select(mono_id().alias('id'), '*')
health_institution_table_DF.printSchema()
health_institution_table_DF.show()

root
 |-- id: long (nullable = false)
 |-- name: string (nullable = true)
 |-- organization: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)

+------------+--------------------+--------------------+-----+-------------------+
|          id|                name|        organization|state|               city|
+------------+--------------------+--------------------+-----+-------------------+
|           0|AMBULATORIO MUNIC...|PREFEITURA MUNICI...|   MA|    POCAO DE PEDRAS|
|           1|CEADIM DE SERRANO...|MUNICIPIO DE SERR...|   MA|SERRANO DO MARANHAO|
|  8589934592|CENTRO DE ATENDIM...|PREFEITURA MUNICI...|   MA|      VARGEM GRANDE|
| 17179869184|CENTRO DE ESPECIA...|PREFEITURA MUNICI...|   MA|    BARAO DE GRAJAU|
| 25769803776|CENTRO DE ESPECIA...|PREFEITURA MUNICI...|   MA|           PINHEIRO|
| 34359738368|CENTRO DE SAUDE A...|MUNICIPIO DE URBA...|   MA|      URBANO SANTOS|
| 42949672960|CENTRO DE SAUDE A...|PREFEITURA MUNICI...|   MA

In [20]:
# Check data quality
if not check_nulls(health_institution_table_DF, ['id', 'name', 'organization', 'state', 'city'], 0): raise Exception('Null: Health institution table')
if not check_has_content(health_institution_table_DF): raise Exception('No content: Health institution table')

In [21]:
# Write parquet file
write_parquet(health_institution_table_DF, 'health_institution')

Writing health_institution Table DONE.


In [22]:
# Create Category table
vaccines_df.createOrReplaceTempView("category_table_DF")
category_table_DF = spark.sql("""
    SELECT DISTINCT vacina_categoria_codigo AS id,
                    vacina_categoria_nome AS name
            FROM category_table_DF
            ORDER BY name
""")

category_table_DF.printSchema()
category_table_DF.show()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|        Comorbidades|
|  2|        Faixa Etária|
| 11|Pessoas com Defic...|
|  3|Pessoas de 60 ano...|
|  7|     Povos Indígenas|
|  6|Povos e Comunidad...|
|  9|Trabalhadores de ...|
+---+--------------------+



In [23]:
# Check data quality
if not check_nulls(category_table_DF, ['id', 'name'], 0): raise Exception('Null: Category table')
if not check_has_content(category_table_DF): raise Exception('No content: Category table')

In [24]:
# Write parquet file
write_parquet(category_table_DF, 'category')

Writing category Table DONE.


In [25]:
# Create Population Groups table
vaccines_df.createOrReplaceTempView("population_group_table_DF")
population_group_table_DF = spark.sql("""
    SELECT DISTINCT vacina_grupoatendimento_codigo AS id,
                    vacina_grupoatendimento_nome AS name
            FROM population_group_table_DF
        ORDER BY name
""")

population_group_table_DF.printSchema()
population_group_table_DF.show()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)

+----+--------------------+
|  id|                name|
+----+--------------------+
| 905|  Cuidador de Idosos|
| 103|   Diabetes Mellitus|
| 105|Doença Renal Crônica|
| 106|Doenças Cardiovas...|
| 907|       Enfermeiro(a)|
| 909|     Fisioterapeutas|
| 911|Funcionário do Si...|
| 107|Hipertensão de di...|
| 102|          Neoplasias|
| 915|       Nutricionista|
| 926|              Outros|
|1102|Pessoas com Defic...|
| 201|Pessoas de 18 a 6...|
| 301|Pessoas de 60 ano...|
| 202|Pessoas de 65 a 6...|
| 203|Pessoas de 70 a 7...|
| 204|Pessoas de 75 a 7...|
| 205|Pessoas de 80 ano...|
| 701|Povos indígenas e...|
| 917|Profissionais e A...|
+----+--------------------+
only showing top 20 rows



In [26]:
# Check data quality
if not check_nulls(population_group_table_DF, ['id', 'name'], 0): raise Exception('Null: Population group table')
if not check_has_content(population_group_table_DF): raise Exception('No content: Population group table')

In [27]:
# Write parquet file
write_parquet(population_group_table_DF, 'population_group')

Writing population_group Table DONE.


In [28]:
# Create Patient table
vaccines_df.createOrReplaceTempView("patient_table_DF")
patient_table_DF = spark.sql("""
    SELECT DISTINCT paciente_id AS id,
                    paciente_idade AS age,
                    paciente_datanascimento AS birthdate,
                    paciente_enumsexobiologico AS gender,
                    paciente_endereco_nmpais AS country,
                    paciente_endereco_uf AS state,
                    paciente_endereco_nmmunicipio AS city
            FROM patient_table_DF
            ORDER BY id
""")

patient_table_DF.printSchema()
patient_table_DF.show()

root
 |-- id: string (nullable = true)
 |-- age: string (nullable = true)
 |-- birthdate: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)

+--------------------+---+-------------------+------+-------+-----+--------------------+
|                  id|age|          birthdate|gender|country|state|                city|
+--------------------+---+-------------------+------+-------+-----+--------------------+
|001172d8023bf74e8...| 69|1951-05-03 00:00:00|     M| BRASIL|   MA|            SAO LUIS|
|0037915f9d8c82740...| 80|1940-07-14 00:00:00|     F| BRASIL|   MA|          ACAILANDIA|
|003f0c135c70c7f4d...| 82|1938-05-09 00:00:00|     F| BRASIL|   MA|  ITINGA DO MARANHAO|
|00ba3b39629b28be3...| 61|1960-01-21 00:00:00|     F| BRASIL|   MA|      BARRA DO CORDA|
|012c38283b39186dd...| 50|1970-04-03 00:00:00|     M| BRASIL|   MA|       PRIMEIRA CRUZ|
|014d4147b3c1cde08...|

In [30]:
# Check data quality
if not check_nulls(patient_table_DF, ['id', 'age', 'birthdate', 'gender', 'country', 'state', 'city'], 0): raise Exception('Null: Patient table')
if not check_has_content(patient_table_DF): raise Exception('No content: Patient table')

In [31]:
# Write parquet file and get back to Spark:
write_parquet(patient_table_DF, 'patient')

Writing patient Table DONE.


In [32]:
# Get back Health Institution table from parquet
health_institution_table_DF = read_parquet('health_institution')

In [33]:
# Create Patient table and write parquet files
vaccines_df.createOrReplaceTempView("vdf")
health_institution_table_DF.createOrReplaceTempView("hidf")
immunization_table_DF = spark.sql("""
    SELECT DISTINCT vdf.paciente_id AS patient_id,
            hidf.id AS health_institution_id,
            vdf.vacina_categoria_codigo AS category_id,
            vdf.vacina_grupoatendimento_codigo AS population_group_id,
            vdf.vacina_codigo AS vaccines_id,
            vdf.vacina_descricao_dose AS vaccines_dose,
            vdf.vacina_dataaplicacao AS jab_date
        FROM vdf INNER JOIN hidf
        ON vdf.estalecimento_nofantasia == hidf.name
        AND vdf.estabelecimento_razaosocial == hidf.organization
        ORDER BY jab_date
""")

immunization_table_DF.printSchema()

root
 |-- patient_id: string (nullable = true)
 |-- health_institution_id: long (nullable = true)
 |-- category_id: string (nullable = true)
 |-- population_group_id: string (nullable = true)
 |-- vaccines_id: string (nullable = true)
 |-- vaccines_dose: string (nullable = true)
 |-- jab_date: string (nullable = true)



In [34]:
# Check data quality
if not check_nulls(immunization_table_DF, \
        ['patient_id', \
        'health_institution_id', \
        'category_id', \
        'population_group_id', \
        'vaccines_id', \
        'vaccines_dose', \
        'jab_date'], 0): raise Exception('Null: Immunization table')
if not check_has_content(immunization_table_DF): raise Exception('No content: Immunization table')

In [35]:
# Write parquet file and get back to Spark:
write_parquet(immunization_table_DF, 'immunization')
immunization_table_DF = read_parquet('immunization')

Writing immunization Table DONE.


In [36]:
immunization_table_DF.show()

+--------------------+---------------------+-----------+-------------------+-----------+-------------+-------------------+
|          patient_id|health_institution_id|category_id|population_group_id|vaccines_id|vaccines_dose|           jab_date|
+--------------------+---------------------+-----------+-------------------+-----------+-------------+-------------------+
|ef2f0d0cb39dc600f...|         661424963584|          2|                203|         86|      1ª Dose|2021-03-29 21:00:00|
|9784680359770f437...|         369367187456|          2|                203|         86|      1ª Dose|2021-03-29 21:00:00|
|f5b0c254411cbb0de...|        1030792151041|          6|                601|         85|      1ª Dose|2021-03-29 21:00:00|
|949a3f3bcdeee1c5a...|         128849018880|          2|                203|         86|      1ª Dose|2021-03-29 21:00:00|
|77153f4b41c71a705...|         566935683072|          2|                202|         86|      1ª Dose|2021-03-29 21:00:00|
|e58d257e23f5124

### Data dictionary

[Dict](./data-dictionary.json)
```
{
  "vaccines": {
    "description": "Vaccines and suppliers",
    "fields": {
      "id": "Identity field",
      "name": "Vaccine name",
      "supplier": "Laboratory/supplier"
    }
  },
  "health_institution": {
    "description": "Hospitals, Nursing home, Clinics",
    "fields": {
      "id": "Identity field",
      "name": "Institution name",
      "organization": "",
      "state": "Brazilian State",
      "city": "City name"
    }
  },
  "category": {
    "description": "Priority groups",
    "fields": {
      "id": "Identity field",
      "name": "Group name"
    }
  },
  "population_group": {
    "description": "Demograph group (professions, age group, ethnicity)",
    "fields": {
      "id": "Identity field",
      "name": "Group name"
    }
  },
  "patient": {
    "description": "Demograph data (age, city, gender)",
    "fields": {
      "id": "Identity field",
      "age": "Patient age (years)",
      "birthdate": "Birth date",
      "gender": "Male/Female",
      "country": "Country (Brazil)",
      "state": "Brazilian State",
      "city": "City name"
    }
  },
  "imunization": {
    "description": "Dimensions, First | second dose, date",
    "fields": {
      "patient_id": "(FK) Patient Identity",
      "health_institution_id": "(FK) Health institution Identity",
      "category_id": "(FK) Category group Identity",
      "population_group_id": "(FK) Population group Identity",
      "vaccines_id": "(FK) Vaccine Identity",
      "vaccines_dose": "First/Second dose",
      "jab_date": "Vaccine date"
    }
  }
}
```

## Step 5: Complete Project Write Up
Tools:
* Python
* Pandas
* Spark

These tools/technologies are apropriated to manipulate large dataset, processing in paralellized clusters

**ETL script should be run weekly basis**, or whenever the .gov.br update the datasets

How the script would approach the problem differently under the following scenarios:
* If the data was increased by 100x: _Use Spark Clustered to parallel the data load_
* If the pipelines were run on a daily basis by 7am: _Refactor the script to process only new informations. The [.gov.br API](https://dados.gov.br/dataset/covid-19-vacinacao/resource/97a8fbcf-941f-4d2e-91ba-dd467d5bdeac?inner_span=True) could be used to request the delta information_
* If the database needed to be accessed by 100+ people: _Store the parquet files on a cloud data lake_

### Sample queries

In [43]:
vaccines_table_DF = read_parquet('vaccines')

# Get Health institution by vaccines aplication
vaccines_table_DF.createOrReplaceTempView("vdf")
health_institution_table_DF.createOrReplaceTempView("hidf")
immunization_table_DF.createOrReplaceTempView("idf")

health_institution_rank = spark.sql("""
    SELECT 
        CASE
            WHEN (GROUPING(hidf.name) = 1) THEN '# Total Institution'
            ELSE hidf.name
        END AS Institution,
        CASE
            WHEN (GROUPING(vdf.name) = 1) THEN '# Total Vaccines'
            ELSE vdf.name
        END AS Vaccine,
        COUNT(*) AS TOTAL
    FROM idf 
    INNER JOIN hidf ON idf.health_institution_id = hidf.id
    INNER JOIN vdf ON idf.vaccines_id = vdf.id
    GROUP BY CUBE(hidf.name, vdf.name)
    ORDER BY Institution, TOTAL DESC, Vaccine
""").toPandas()

health_institution_rank.style

,Institution,Vaccine,TOTAL
0,# Total Institution,# Total Vaccines,1000
1,# Total Institution,Covid-19-Coronavac-Sinovac/Butantan,833
2,# Total Institution,Vacina Covid-19 - Covishield,167
3,AMBULATORIO MUNICIPAL PRESIDENTE MEDICI,# Total Vaccines,1
4,AMBULATORIO MUNICIPAL PRESIDENTE MEDICI,Covid-19-Coronavac-Sinovac/Butantan,1
5,CEADIM DE SERRANO DO MARANHAO,# Total Vaccines,5
6,CEADIM DE SERRANO DO MARANHAO,Covid-19-Coronavac-Sinovac/Butantan,3
7,CEADIM DE SERRANO DO MARANHAO,Vacina Covid-19 - Covishield,2
8,CENTRO DE ATENDIMENTO A CRIANCA CAMILA BIZOTO,# Total Vaccines,1
9,CENTRO DE ATENDIMENTO A CRIANCA CAMILA BIZOTO,Covid-19-Coronavac-Sinovac/Butantan,1


In [44]:
# Get vaccines aplication
vaccines_rank = spark.sql("""
    SELECT 
        CASE
            WHEN (GROUPING(vdf.supplier) = 1) THEN '# Total Supplier'
            ELSE vdf.supplier
        END AS Supplier,
        CASE
            WHEN (GROUPING(vdf.name) = 1) THEN '# Total Vaccines'
            ELSE vdf.name
        END AS Vaccine,
        COUNT(*) AS TOTAL
    FROM idf 
    INNER JOIN vdf ON idf.vaccines_id = vdf.id
    GROUP BY CUBE(vdf.supplier, vdf.name)
    ORDER BY Supplier, TOTAL DESC
""").toPandas()

vaccines_rank.style

,Supplier,Vaccine,TOTAL
0,# Total Supplier,# Total Vaccines,1000
1,# Total Supplier,Covid-19-Coronavac-Sinovac/Butantan,833
2,# Total Supplier,Vacina Covid-19 - Covishield,167
3,FUNDACAO BUTANTAN,# Total Vaccines,833
4,FUNDACAO BUTANTAN,Covid-19-Coronavac-Sinovac/Butantan,833
5,FUNDACAO OSWALDO CRUZ,# Total Vaccines,167
6,FUNDACAO OSWALDO CRUZ,Vacina Covid-19 - Covishield,167
